In [ ]:
from datasets import Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
import pandas as pd
import random
data_frame = pd.read_csv('/kaggle/input/custom-data-for-fashion-chatbot/dataset.csv')

tokenizer = GPT2Tokenizer.from_pretrained('NlpHUST/gpt2-vietnamese')
tokenizer.pad_token = '<pad>'
tokenizer.pad_token_id = 1

questions = data_frame.question
answers = data_frame.answer

texts = [f"Hỏi: {q} Trả lời: {a} |" for q, a in zip(questions, answers)]
random.shuffle(texts)
dataset = Dataset.from_dict({'text': texts})

def tokenize_function(examples):
    return tokenizer(examples['text'],return_tensors="pt",padding="max_length", truncation=True, max_length=768)

dataset = dataset.map(tokenize_function, batched=True)
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])
train_dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/1168 [00:00<?, ? examples/s]

In [ ]:
print(len(questions), len(answers))
print(texts[70])

In [ ]:
print(tokenizer.decode(torch.tensor(500)))
# print(dataset['input_ids'][80])
print(tokenizer.encode('[SEP]'))

In [ ]:

model = GPT2LMHeadModel.from_pretrained('NlpHUST/gpt2-vietnamese')
# model = GPT2LMHeadModel.from_pretrained('minhtoan/vietnamese-gpt2-finetune')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Cấu hình optimizer
optimizer = AdamW(model.parameters(), lr=1e-4)


In [55]:
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)


In [ ]:
from tqdm import tqdm  # Để hiển thị tiến trình
optimizer = AdamW(model.parameters(), lr=1e-6)

num_epochs = 1
for epoch in range(num_epochs):
    model.train() 
    total_loss = 0
    
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}", leave=False):
        inputs = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)  # Lấy attention_mask
        labels = inputs.clone()
        # labels[:, :-1] = inputs[:, 1:].clone()  # Dịch các token đi 1 bước
        # labels[:, -1] = tokenizer.pad_token_id  # Gán token pad cho token cuối cùng, vì không có token tiếp theo
        optimizer.zero_grad()
        outputs = model(inputs, labels=labels,attention_mask=attention_mask)
        loss = outputs.loss
        loss.backward()
        # Cập nhật tham số
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_dataloader)}")


Epoch 1, Loss: 0.42006354850448974


In [ ]:
text = 'Chọn đồ gì đi ăn đám cưới đây';
input_text = f"Hỏi: {text} Trả lời:"
inputs = tokenizer(input_text, return_tensors="pt").to(device)

model.eval()
outputs = model.generate(
    inputs['input_ids'],
    pad_token_id=tokenizer.pad_token_id,
    do_sample=True,
    max_length=128,
    min_length=10,
    top_k=40,
    num_beams=5,
    early_stopping=True,
    no_repeat_ngram_size=2,
    num_return_sequences=1
)


generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
for i, text in enumerate(generated_texts):
    text = text.split('|')[0]
    text = text.split('.')
    text = text[:2]
    text = [t for t in text if 'http' not in t]
    text = '.'.join(text)
    print(f"Generated Text {i+1}:\n{text}.")


Generated Text 1:
Hỏi: Chọn đồ gì đi ăn đám cưới đây Trả lời: Hãy chọn trang phục đơn giản, nhẹ nhàng như áo thun, quần jeans, giày bệt hoặc giày cao gót để tạo cảm giác thoải mái khi đi dự tiệc. Màu sắc phù hợp là những gam màu trung tính như đen, xám hoặc trắng.


In [ ]:
# Lưu mô hình sau khi huấn luyện
model.save_pretrained('./trained_gpt2_vietnamese')
tokenizer.save_pretrained('./trained_gpt2_vietnamese')

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GPT2LMHeadModel.from_pretrained('/kaggle/input/gpt-fineturing')
model.to(device)